In [1]:
# Import necessary libraries
import import_ipynb
# import exploration
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.utils import resample
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xgb 

## Merging the two datasets base on their Account ID

In [2]:

def merge_datasets(dataset1_path, dataset2_path, merge_column):
    # Load the datasets
    df1 = pd.read_csv(dataset1_path)
    df2 = pd.read_csv(dataset2_path)
    
    # Merge the datasets based on the specified column
    merged_df = pd.merge(df1, df2, on=merge_column, how='inner')
    
    return merged_df

# Example usage:
dataset1_path = "../data/orders_data_competition.csv"
dataset2_path = "../data/clients_data_competition.csv"
merge_column = "Account ID"

df = merge_datasets(dataset1_path, dataset2_path, merge_column)
df.sample(10)


,Order ID,Account ID,Market Key,Security ID,Order Type,Order Time,Order Via,Is Completed,Is Canceled,Expire Date,...,Gender,Risk Rate,Company Name,Is Closed,Is Dormant,Is Profile Suspended,Is Client Suspended,Client Type Name,OpenDate,BirthDate
1243243,1243243,6536,Egypt,42,Buy,2023-06-21 11:14:17.000,Online,0,0,2023-06-21 00:00:00.000,...,Male,Low,HSB,0,0.0,0,0,Individuals,12/18/2011,1951-04-27
1777057,1777057,5759,Egypt,144,Sell,2023-12-25 10:29:28.000,Online,1,0,2023-12-25 00:00:00.000,...,Male,Not Assigned,HSB,0,0.0,0,0,Individuals,11/16/2023,1971-11-03
1409129,1409129,4614,Egypt,52,Buy,2023-09-11 12:49:21.000,Online,0,0,2023-09-11 00:00:00.000,...,Male,Not Assigned,HSB,0,0.0,0,0,Individuals,12/29/2022,1945-01-15
83189,83189,5794,Egypt,28,Sell,2024-02-13 10:56:08.000,Online,0,0,2024-02-13 00:00:00.000,...,Male,Medium,HSB,0,0.0,0,0,Individuals,7/6/2011,1979-12-12
1431722,1431722,4285,Egypt,5,Sell,2023-09-20 11:13:16.000,Online,1,0,2023-09-20 00:00:00.000,...,Male,Medium,HSB,0,0.0,0,0,Individuals,8/11/2014,1946-05-05
383518,383518,10872,Egypt,66,Buy,2022-04-26 11:08:39.000,Online,1,0,2022-04-26 00:00:00.000,...,Female,Medium,HSB,0,0.0,0,0,Individuals,3/31/2020,1966-04-03
1123859,1123859,5556,Egypt,82,Sell,2023-05-09 13:48:03.000,Online,0,0,2023-05-09 00:00:00.000,...,Male,Low,HSB,0,0.0,0,0,Individuals,7/5/2011,1998-06-03
220911,220911,7802,Egypt,33,Sell,2022-01-03 10:23:31.000,Online,1,0,2022-01-03 00:00:00.000,...,Female,Low,HSB,0,-1.0,1,0,Individuals,9/17/2018,1947-02-15
698339,698339,160,Egypt,126,Sell,2022-11-10 14:14:54.000,Online,0,1,2022-11-10 00:00:00.000,...,Male,Low,HSB,0,0.0,0,0,Individuals,2/18/2009,1978-08-17
336879,336879,12646,Egypt,124,Sell,2022-03-21 11:54:47.000,Online,1,0,2022-03-21 00:00:00.000,...,Male,Medium,HSB,0,0.0,0,0,Individuals,12/6/2020,1984-05-14


In [3]:
# Check for missing values
print("\nMissing values in the dataset:")
df.isnull().sum()



Missing values in the dataset:


Order ID                0
Account ID              0
Market Key              0
Security ID             0
Order Type              0
Order Time              0
Order Via               0
Is Completed            0
Is Canceled             0
Expire Date             6
Execution Status        1
quantity                0
Price                   0
Sector Name             0
Executed Quantity       0
Quantity                0
Client ID               0
Gender                  0
Risk Rate               0
Company Name            0
Is Closed               0
Is Dormant              0
Is Profile Suspended    0
Is Client Suspended     0
Client Type Name        0
OpenDate                0
BirthDate               0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
df.nunique()

Order ID                1987936
Account ID                 9727
Market Key                    1
Security ID                 335
Order Type                    2
Order Time              1749453
Order Via                     4
Is Completed                  2
Is Canceled                   2
Expire Date                 828
Execution Status              3
quantity                  32994
Price                     20033
Sector Name                  23
Executed Quantity         28705
Quantity                  32994
Client ID                  8751
Gender                        2
Risk Rate                     4
Company Name                  1
Is Closed                     2
Is Dormant                    2
Is Profile Suspended          2
Is Client Suspended           2
Client Type Name              2
OpenDate                   3260
BirthDate                  7631
dtype: int64

In [6]:
def drop_null_rows(df):
    # Drop rows with null values
    df.dropna(inplace=True)
    return df

# Example usage:
# Assuming df is your DataFrame
df = drop_null_rows(df)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1987936 entries, 0 to 1987942
Data columns (total 27 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Order ID              int64  
 1   Account ID            int64  
 2   Market Key            object 
 3   Security ID           int64  
 4   Order Type            object 
 5   Order Time            object 
 6   Order Via             object 
 7   Is Completed          int64  
 8   Is Canceled           int64  
 9   Expire Date           object 
 10  Execution Status      object 
 11  quantity              int64  
 12  Price                 float64
 13  Sector Name           object 
 14  Executed Quantity     int64  
 15  Quantity              int64  
 16  Client ID             int64  
 17  Gender                object 
 18  Risk Rate             object 
 19  Company Name          object 
 20  Is Closed             int64  
 21  Is Dormant            float64
 22  Is Profile Suspended  int64  
 23  Is Client Su

In [8]:
def remove_unnecessary_attributes(data, attributes_to_remove):
    """Remove unnecessary attributes from the dataset."""
    # Remove attributes with only one unique value
    data = data.loc[:, data.nunique() > 1]

    # Remove attributes specified in attributes_to_remove
    columns_to_remove = [col for col in attributes_to_remove if col in data.columns]
    data.drop(columns=columns_to_remove, inplace=True)

    return data

# Example usage:
attributes_to_remove = ["Order ID", "Account ID", "Security ID", "Client ID","Order Time", "OpenDate", "Quantity", "BirthDate"]
df = remove_unnecessary_attributes(df, attributes_to_remove)

C:\Users\Mohamed\AppData\Local\Temp\ipykernel_18672\3983029155.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns=columns_to_remove, inplace=True)


In [9]:
df.nunique()

Order Type                  2
Order Via                   4
Is Completed                2
Is Canceled                 2
Expire Date               828
Execution Status            3
quantity                32994
Price                   20033
Sector Name                23
Executed Quantity       28705
Gender                      2
Risk Rate                   4
Is Closed                   2
Is Dormant                  2
Is Profile Suspended        2
Is Client Suspended         2
Client Type Name            2
dtype: int64

In [10]:

# Assuming 'df' is your DataFrame
df["Price"] = df["Price"].astype(int)

# Change data types
df["Is Completed"] = df["Is Completed"].astype(bool)
df["Is Canceled"] = df["Is Canceled"].astype(bool)
df["Is Closed"] = df["Is Closed"].astype(bool)
df["Is Profile Suspended"] = df["Is Profile Suspended"].astype(bool)
df["Is Client Suspended"] = df["Is Client Suspended"].astype(bool)
df["Is Dormant"] = df["Is Dormant"].astype(bool)
df['Order Type'] = df['Order Type'].astype(bool)
df["Gender"] = df["Gender"].astype(bool)
df["Risk Rate"] = df["Risk Rate"].astype(bool)

# Convert 'Order Type', 'Order Time', 'Order Via', 'Expire Date', 'Execution Status', 'Sector Name', 'Client Type Name' to category if they have limited unique values
df["Order Via"] = df["Order Via"].astype("category")
df["Expire Date"] = pd.to_datetime(df["Expire Date"])
df["Execution Status"] = df["Execution Status"].astype("category")
df["Sector Name"] = df["Sector Name"].astype("category")
df["Client Type Name"] = df["Client Type Name"].astype("category")

# Convert categorical variables to codes
df['Order Via'] = df['Order Via'].cat.codes
df['Execution Status'] = df['Execution Status'].cat.codes
df['Sector Name'] = df['Sector Name'].cat.codes
df['Client Type Name'] = df['Client Type Name'].cat.codes

# Add Year and Month of Expire date and remove the date itself
df["Expire Year"] = pd.to_datetime(df["Expire Date"]).dt.year
df["Expire Month"] = pd.to_datetime(df["Expire Date"]).dt.month
df = remove_unnecessary_attributes(df, ["Expire Date"])
df["Expire Year"] = df["Expire Year"].astype("category")
df["Expire Month"] = df["Expire Month"].astype("category")
df['Expire Year'] = df['Expire Year'].cat.codes
df['Expire Month'] = df['Expire Month'].cat.codes




C:\Users\Mohamed\AppData\Local\Temp\ipykernel_18672\3983029155.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns=columns_to_remove, inplace=True)


In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1987936 entries, 0 to 1987942
Data columns (total 15 columns):
 #   Column                Dtype
---  ------                -----
 0   Order Via             int8 
 1   Is Completed          bool 
 2   Is Canceled           bool 
 3   Execution Status      int8 
 4   quantity              int64
 5   Price                 int32
 6   Sector Name           int8 
 7   Executed Quantity     int64
 8   Is Closed             bool 
 9   Is Dormant            bool 
 10  Is Profile Suspended  bool 
 11  Is Client Suspended   bool 
 12  Client Type Name      int8 
 13  Expire Year           int8 
 14  Expire Month          int8 
dtypes: bool(6), int32(1), int64(2), int8(6)
memory usage: 75.8 MB


In [12]:

def undersample_tomek_links(df, target_column):
    """
    Undersample the majority class using Tomek links.
    
    Parameters:
    - df: DataFrame containing the dataset.
    - target_column: Name of the column representing the target variable.
    
    Returns:
    - balanced_df: DataFrame with balanced classes.
    """
    # Separate data into features and target variable
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Undersample using Tomek links
    tl = TomekLinks()
    X_resampled, y_resampled = tl.fit_resample(X, y)

    # Combine resampled data into DataFrame
    balanced_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name=target_column)], axis=1)

    return balanced_df

# Example usage:
df = undersample_tomek_links(df, "Is Closed")


In [13]:
def save_preprocessed_dataset(df, file_path):
    """
    Save preprocessed DataFrame to a CSV file.
    
    Parameters:
    - df: Preprocessed DataFrame.
    - file_path: File path to save the CSV file.
    """
    df.to_csv(file_path, index=False)
    print(f"Preprocessed dataset saved to {file_path}")

# Example usage:
# Assuming df is your preprocessed DataFrame and file_path is the desired file path
file_path = "../data/preprocessed_dataset.csv"
save_preprocessed_dataset(df, file_path)


Preprocessed dataset saved to ../data/preprocessed_dataset.csv
